In [ ]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
diabetes = pd.read_csv('data/pima-indians-diabetes.csv')

In [ ]:
diabetes.head()

In [ ]:
diabetes.columns

In [ ]:
cols_to_norm = ['Number_pregnant', 'Glucose_concentration', 'Blood_pressure', 'Triceps',
       'Insulin', 'BMI', 'Pedigree']

In [ ]:
diabetes[cols_to_norm] = diabetes[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max()-x.min() ))

In [ ]:
diabetes.head()

In [ ]:
num_preg = tf.feature_column.numeric_column('Number_pregnant')
plasma_gluv = tf.feature_column.numeric_column('Glucose_concentration')
dias_press = tf.feature_column.numeric_column('Blood_pressure')
tricep = tf.feature_column.numeric_column('Triceps')
insulin = tf.feature_column.numeric_column('Insulin')
bmi = tf.feature_column.numeric_column('BMI')
diabetes_pedigree = tf.feature_column.numeric_column('Pedigree')
age = tf.feature_column.numeric_column('Age')

In [ ]:
assigned_group = tf.feature_column.categorical_column_with_vocabulary_list('Group',['A','B','C','D'])

In [ ]:
#assigned_group = tf.feature_column.categorical_column_with_hash_bucket('Group',hash_bucket_size=10)

In [ ]:
diabetes['Age'].hist(bins=20)

In [ ]:
age_bucket = tf.feature_column.bucketized_column(age,boundaries=[20,30,40,50,60,70,80])

In [ ]:
feat_cols = [num_preg,plasma_gluv,dias_press,tricep,insulin,bmi,diabetes_pedigree,assigned_group,age_bucket]

In [ ]:
feat_cols

In [ ]:
# Train and test split

In [ ]:
x_data = diabetes.drop('Class',axis=1)

In [ ]:
x_data.head()

In [ ]:
labels = diabetes['Class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_data,labels,test_size = 0.3)

In [ ]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train,batch_size=10,num_epochs=1000,shuffle=True)

In [ ]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols,n_classes=2)

In [ ]:
model.train(input_fn=input_func,steps=1000)

In [ ]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train,batch_size=10,num_epochs=1,shuffle=False)

In [ ]:
results = model.evaluate(eval_input_func)

In [ ]:
results

In [ ]:
## Predictions

In [ ]:
pred_input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,batch_size=10,num_epochs=1,shuffle=False)

In [ ]:
predictions = model.predict(pred_input_func)

In [ ]:
my_pred = list(predictions)

In [ ]:
my_pred

In [ ]:
dnn_model = tf.estimator.DNNClassifier(hidden_units=[10,10,10],feature_columns=feat_cols,n_classes=2)

In [ ]:
embedded_group_col = tf.feature_column.embedding_column(assigned_group,dimension=4)

In [ ]:
feat_cols = [num_preg,plasma_gluv,dias_press,tricep,insulin,bmi,diabetes_pedigree,embedded_group_col,age_bucket]

In [ ]:
input_func = tf.estimator.inputs.pandas_input_fn(X_train, y_train,batch_size=10,num_epochs=1000,shuffle=True)

In [ ]:
dnn_model = tf.estimator.DNNClassifier(hidden_units=[10,10,10],feature_columns=feat_cols,n_classes=2)

In [ ]:
dnn_model.train(input_fn=input_func,steps=1000)

In [ ]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test, y=y_test,batch_size=10,num_epochs=1,shuffle=False)

In [ ]:
dnn_model.evaluate(eval_input_func)